<a href="https://colab.research.google.com/github/SIDIBEMoussa/Muter-automatiquement-bad-words-dans-les-audios-ou-vid-os/blob/main/muter_bad_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-thkldi4w
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-thkldi4w
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━

In [2]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe("movie.mp4", word_timestamps=True)

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 111MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [3]:
result['segments']

[{'id': 0,
  'seek': 0,
  'start': np.float64(0.0),
  'end': np.float64(5.4),
  'text': " J'ai lu que la rémunération horaire pour un avocat établi à Genève était d'au moins 100 francs supérieurs à ce qui se pratique à Lausanne.",
  'tokens': [50364,
   508,
   6,
   1301,
   10438,
   631,
   635,
   3960,
   23717,
   27901,
   2569,
   9020,
   2016,
   517,
   1305,
   905,
   267,
   4823,
   455,
   2081,
   1531,
   3632,
   31397,
   11806,
   274,
   6,
   1459,
   13099,
   2319,
   30514,
   82,
   9331,
   10746,
   2156,
   1531,
   1769,
   1956,
   369,
   43740,
   1531,
   2369,
   301,
   12674,
   13,
   50636],
  'temperature': 0.0,
  'avg_logprob': -0.20696717182188543,
  'compression_ratio': 1.525,
  'no_speech_prob': 0.1608119010925293,
  'words': [{'word': ' J',
    'start': np.float64(0.0),
    'end': np.float64(0.58),
    'probability': np.float64(0.36042463779449463)},
   {'word': "'ai",
    'start': np.float64(0.58),
    'end': np.float64(0.62),
    'probabi

In [7]:
bad_words = {"drogue", "shit", "bitch", "damn", "asshole", "bastard", "dick"}

In [8]:
profanity_timestamps = []

for seg in result["segments"]:
    for word in seg["words"]:
        if word["word"].strip().lower() in bad_words:
            profanity_timestamps.append((word["start"], word["end"]))

In [9]:
profanity_timestamps

[(np.float64(146.16), np.float64(146.6))]

In [14]:
import subprocess

def mute_profanities(input_video, output_video, profanity_timestamps):
    # Construire le filtre audio
    filters = []
    for start, end in profanity_timestamps:
        filters.append(f"volume=enable='between(t,{start},{end})':volume=0")

    volume_filter = ",".join(filters)

    # Commande ffmpeg
    command = [
        "ffmpeg",
        "-i", input_video,
        "-af", volume_filter,
        "-c:v", "copy",  # on ne touche pas à la vidéo
        "-y",  # overwrite
        output_video
    ]

    print("Running FFmpeg command:")
    print(" ".join(command))

    subprocess.run(command)

In [16]:
mute_profanities(
    input_video="movie.mp4",
    output_video="clean_movie.mp4",
    profanity_timestamps=profanity_timestamps
)

Running FFmpeg command:
ffmpeg -i movie.mp4 -af volume=enable='between(t,146.16,146.6)':volume=0 -c:v copy -y clean_movie.mp4
